In [1]:
import os
import glob
import numpy as np
import pandas as pd
import datetime
import shutil


pd.set_option("display.max_colwidth", 200)

In [2]:
def date_to_sortable(date):
    """Data in format: 2022Nov07
    """
    year, month, day = date[:4], date[4:7], date[7:]
    dictmon = {
        "Sep": "09",
        "Oct": "10",
        "Nov": "11",
    }
    return "-".join([year, dictmon[month], day])

In [3]:
ddir = '/bigstore/Images2022/Gaby/dredFISH'
outdir = '/bigstore/GeneralStorage/Data/dredFISH'

In [4]:
datasets = np.sort(glob.glob("DPNMF*", dir_fd=ddir))
datasets

array(['DPNMF-FR_7C,PFA+Methanol,PFA,Methanol_2022Nov07',
       'DPNMF-FR_7C_,PFA+Methanol,PFA,Methanol_2022Nov04',
       'DPNMF-FR_7C_,PFA+Methanol,PFA,Methanol_2022Nov07',
       'DPNMF-FR_R1_4A_UC_R2_5C_2022Nov27',
       'DPNMF-FR_Z1,Z2,9A,Z3,Z4,6C_2022Nov15', 'DPNMF-R_8C_2022Oct18',
       'DPNMF-TL-R1_4A_UC-BL-R2_5C_2022Nov21',
       'DPNMF-TL-R1_4A_UC-BL-R2_5C_2022Nov22',
       'DPNMF_uncleared_2022Sep20', 'DPNMF_uncleared_2022Sep21'],
      dtype='<U48')

In [5]:
res = []
for dst in datasets:
    date = date_to_sortable(dst.split('_')[-1])
    
    p = os.path.join(ddir, dst) 
    a = glob.glob("fishdata*", dir_fd=p)
    a.sort(key=lambda x: os.path.getmtime(os.path.join(p, x)))
    
    if len(a) > 0:
        picked = a[-1]
    else:
        picked = ""
        
    p = os.path.join(ddir, dst, picked) 
    b1 = glob.glob("*_metadata.csv", dir_fd=p)
    b1.sort(key=lambda x: os.path.getmtime(os.path.join(p, x)))
    
    b2 = glob.glob("*_matrix.csv", dir_fd=p)
    b2.sort(key=lambda x: os.path.getmtime(os.path.join(p, x)))
    
    res.append({
        'date': date,
        'dataset': dst,
        'fishdata-all': a,
        'fishdata-picked': picked,
        'n_metadata_csv': len(b1),
        'n_matrix_csv': len(b2),
    })
    
    # print(date, dst, end=":\n")
    # print(a)
res = pd.DataFrame(res).sort_values(['date', 'dataset'])
res

,date,dataset,fishdata-all,fishdata-picked,n_metadata_csv,n_matrix_csv
8,2022-09-20,DPNMF_uncleared_2022Sep20,[],,0,0
9,2022-09-21,DPNMF_uncleared_2022Sep21,[],,0,0
5,2022-10-18,DPNMF-R_8C_2022Oct18,"[fishdata_without_strip, fishdata_with_strip, fishdata_with_strip_no_highpass, fishdata_stitched_correction, fishdata_stitched, fishdata_test, fishdata_2022Nov01, fishdata_2022Nov03, fishdata_2022...",fishdata,4,4
1,2022-11-04,"DPNMF-FR_7C_,PFA+Methanol,PFA,Methanol_2022Nov04","[fishdata_universal_strip, fishdata_2022Nov03]",fishdata_2022Nov03,0,0
0,2022-11-07,"DPNMF-FR_7C,PFA+Methanol,PFA,Methanol_2022Nov07","[fishdata_2022Nov08, fishdata_2022Nov15]",fishdata_2022Nov15,10,10
2,2022-11-07,"DPNMF-FR_7C_,PFA+Methanol,PFA,Methanol_2022Nov07",[],,0,0
4,2022-11-15,"DPNMF-FR_Z1,Z2,9A,Z3,Z4,6C_2022Nov15",[fishdata_2022Nov15],fishdata_2022Nov15,16,16
6,2022-11-21,DPNMF-TL-R1_4A_UC-BL-R2_5C_2022Nov21,[],,0,0
7,2022-11-22,DPNMF-TL-R1_4A_UC-BL-R2_5C_2022Nov22,[],,0,0
3,2022-11-27,DPNMF-FR_R1_4A_UC_R2_5C_2022Nov27,[fishdata_2022Nov15],fishdata_2022Nov15,9,9


In [6]:
resundone = res[res['n_metadata_csv']==0]
resundone

,date,dataset,fishdata-all,fishdata-picked,n_metadata_csv,n_matrix_csv
8,2022-09-20,DPNMF_uncleared_2022Sep20,[],,0,0
9,2022-09-21,DPNMF_uncleared_2022Sep21,[],,0,0
1,2022-11-04,"DPNMF-FR_7C_,PFA+Methanol,PFA,Methanol_2022Nov04","[fishdata_universal_strip, fishdata_2022Nov03]",fishdata_2022Nov03,0,0
2,2022-11-07,"DPNMF-FR_7C_,PFA+Methanol,PFA,Methanol_2022Nov07",[],,0,0
6,2022-11-21,DPNMF-TL-R1_4A_UC-BL-R2_5C_2022Nov21,[],,0,0
7,2022-11-22,DPNMF-TL-R1_4A_UC-BL-R2_5C_2022Nov22,[],,0,0


In [7]:
resdone = res[res['n_metadata_csv']!=0]
resdone

,date,dataset,fishdata-all,fishdata-picked,n_metadata_csv,n_matrix_csv
5,2022-10-18,DPNMF-R_8C_2022Oct18,"[fishdata_without_strip, fishdata_with_strip, fishdata_with_strip_no_highpass, fishdata_stitched_correction, fishdata_stitched, fishdata_test, fishdata_2022Nov01, fishdata_2022Nov03, fishdata_2022...",fishdata,4,4
0,2022-11-07,"DPNMF-FR_7C,PFA+Methanol,PFA,Methanol_2022Nov07","[fishdata_2022Nov08, fishdata_2022Nov15]",fishdata_2022Nov15,10,10
4,2022-11-15,"DPNMF-FR_Z1,Z2,9A,Z3,Z4,6C_2022Nov15",[fishdata_2022Nov15],fishdata_2022Nov15,16,16
3,2022-11-27,DPNMF-FR_R1_4A_UC_R2_5C_2022Nov27,[fishdata_2022Nov15],fishdata_2022Nov15,9,9


# Save results out

In [8]:
outdir

'/bigstore/GeneralStorage/Data/dredFISH'

In [10]:
for idx, res in resdone.iterrows():
    dst = res['dataset']
    dst_out = dst.replace(',', '_')
    picked = res['fishdata-picked']
    
    # ...
    _indir = os.path.join(ddir, dst, picked) 
    b1 = glob.glob("*_metadata.csv", dir_fd=_indir)
    b1.sort(key=lambda x: os.path.getmtime(os.path.join(_indir, x)))
    
    b2 = glob.glob("*_matrix.csv", dir_fd=_indir)
    b2.sort(key=lambda x: os.path.getmtime(os.path.join(_indir, x)))
    
    # create dataset dir (and 'orig' sub dir)
    _outdir = os.path.join(outdir, dst_out)
    if not os.path.isdir(_outdir):
        os.mkdir(_outdir)
        
    _outdir = os.path.join(outdir, dst_out, 'orig')
    if not os.path.isdir(_outdir):
        os.mkdir(_outdir)
        
    for b in np.hstack([b1, b2]):
        src = os.path.join( _indir, b)
        dst = os.path.join(_outdir, b.replace(',', '_'))
        print(b.replace(',', '_'))
        shutil.copy(src, dst)
        

DPNMF-R_8C_2022Oct18_Section4_metadata.csv
DPNMF-R_8C_2022Oct18_Section1_metadata.csv
DPNMF-R_8C_2022Oct18_Section3_metadata.csv
DPNMF-R_8C_2022Oct18_Section2_metadata.csv
DPNMF-R_8C_2022Oct18_Section4_matrix.csv
DPNMF-R_8C_2022Oct18_Section1_matrix.csv
DPNMF-R_8C_2022Oct18_Section3_matrix.csv
DPNMF-R_8C_2022Oct18_Section2_matrix.csv
DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section1_metadata.csv
DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section10_metadata.csv
DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section15_metadata.csv
DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section16_metadata.csv
DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section19_metadata.csv
DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section2_metadata.csv
DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section20_metadata.csv
DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section3_metadata.csv
DPNMF-FR_7C_PFA+Methanol_PFA_Methanol_2022Nov07_Section4_metadata.csv
DPNMF-FR_7C_PFA+Methanol_PFA_